In [ ]:
# import common bib
import numpy as np
import pandas as pd

In [ ]:
#import bib for work on google docx
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import dataset and cut priority info
#put ur dataset url in wb ()
#put ur worksheet name in ws ()
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1n0WwmvYFt0_kdvf4wkBepjKDC1j5Y58jKxb-b5LkeX8/edit#gid=506878057')
ws = wb.worksheet('RESULTS.CSV')
rows = ws.get_all_values()

ds = pd.DataFrame.from_records(rows)

ds.columns = ds.loc[6]
ds = ds.loc[7:]

In [ ]:
#parsing url for SMILE 
from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

In [ ]:
# add SMILE column in original table 
ds['Smiles'] = ds['Library/ID'].apply(lambda x: CIRconvert(x))

In [ ]:
# create a dictionary with types of organic compounds 
# the order affects the main join class in the new column
Classes_Org = {
               'c3' : 'Three_Aroma',
               'c2' : 'Bi_Aroma',
               'c1' : 'Aroma',
               'C3' : 'Three_Cyclo', 
               'C2' : 'Bi_Cyclo',
               'C1' : 'Cyclo', 
               '#': 'Alkyne',
               'c' : 'Aroma',
               '=' : 'Alkene',
               'S': 'S',
               'N' : 'N',
               '(C' : 'ISO',
               'C' : 'Alkane',
               }

In [ ]:
#a function that iterates over the dictionary and extracts the main connection class from it 
#(the class that is in dictionary priority)
def types_of(mol):
  result = []
  for i in Classes_Org:
    if mol.find(i) != -1 and Classes_Org[i] not in result:
      result.append(Classes_Org[i])
  return result[0]

In [ ]:
#create type of organic compaund column in original table
ds['type'] = ds['Smiles'].apply(lambda x : types_of(x))

In [ ]:
#check new column
print(ds['type'])

7      Alkene
8      Alkene
9         ISO
10     Alkane
11        ISO
        ...  
174    Alkane
175    Alkane
176    Alkane
177    Alkane
178    Alkane
Name: type, Length: 172, dtype: object


In [ ]:
#download table
#ds.to_excel(r'C:\mydata.xlsx')

In [ ]:
#check all table
display(ds)

6,Header=,PK,RT,Area Pct,Library/ID,Ref,CAS,Qual,Smiles,type
7,1=,1,1.4770,0.0056,"Butanal, 2-methyl-",1943,000096-17-3,9,CCC(C)C=O,Alkene
8,2=,2,1.5307,0.0577,"Octanamide, N-(2-butyl)-N-methyl-",84220,1000458-72-7,9,CCCCCCCC(=O)N(C)C(C)CC,Alkene
9,3=,3,1.7292,0.3673,"Butane, 2-methyl-",804,000078-78-4,45,CCC(C)C,ISO
10,4=,4,1.8330,0.2918,Pentane,800,000109-66-0,56,CCCCC,Alkane
11,5=,5,2.0456,0.0308,"Butane, 2,2-dimethyl-",2032,000075-83-2,83,CCC(C)(C)C,ISO
...,...,...,...,...,...,...,...,...,...,...
174,168=,168,25.4308,0.0743,Pentadecane,83511,000629-62-9,97,CCCCCCCCCCCCCCC,Alkane
175,169=,169,26.6823,0.0377,Hexadecane,98285,000544-76-3,96,CCCCCCCCCCCCCCCC,Alkane
176,170=,170,27.3227,0.0210,Hexacosane,242228,0630-01-03,83,CCCCCCCCCCCCCCCCCCCCCCCCCC,Alkane
177,171=,171,28.1685,0.0533,Heptadecane,112528,000629-78-7,96,CCCCCCCCCCCCCCCCC,Alkane
